In [1]:
# Importing the necessary librarires
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import mysql.connector
from mysql.connector import Error

In [2]:
def scrape_profile(url):
    # Set up Selenium Options
    options = Options()
    options.headless = True # Run the Browser in headless mode

    # Set up the Selenium Chrome Driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Open the Twitter Profile url
    driver.get(url)
    time.sleep(2) # Wait for the page to load

    # Extract the required details
    try:
        username = driver.find_element(By.XPATH, "//div[@data-testid='UserName']")
    except NoSuchElementException:
        username = None
    try:
        bio = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']")
    except NoSuchElementException:
        bio = None
    try:
        following_count = driver.find_element(By.XPATH, "//a[contains(@href,'/following')]//span[1]")
    except NoSuchElementException:
        following_count = None
    try:
        followers_count = driver.find_element(By.XPATH, "//a[contains(@href,'/followers')]//span[1]")
    except NoSuchElementException:
        followers_count = None
    try:
        location = driver.find_element(By.XPATH, "//span[@data-testid='UserLocation']")
    except NoSuchElementException:
        location = None
    try:
        website = driver.find_element(By.XPATH, "//a[@data-testid='UserUrl']")
    except NoSuchElementException:
        website = None
    
    print(username.text if username else 'N/A')
            
    # Calling insert_data_into_sql function to insert scraped data into SQL Database
    insert_data_into_sql(username.text if username else 'N/A',
                         bio.text if bio else 'N/A',
                         following_count.text if following_count else 'N/A',
                         followers_count.text if followers_count else 'N/A',
                         location.text if location else 'N/A',
                         website.text if website else 'N/A')
    
    # Close the Selenium Chrome Driver
    driver.quit()

In [3]:
def insert_data_into_sql(username, bio, following_count, followers_count, location, website):
    try:
        # Connect to the MySQL server
        db = mysql.connector.connect(host = "127.0.0.1",
                                     user = "root",
                                     password = "root",
                                     database = "profiles")
        mycursor = db.cursor()
        # Prepare the sql query to insert the data
        query = "INSERT INTO twitter_profiles (username, bio, following_count, followers_count, location, website) VALUES (%s, %s, %s, %s, %s, %s)"
        values = (username, bio, following_count, followers_count, location, website)

        # Execute the SQL Query
        mycursor.execute(query, values)
        
        # Commit the changes to the database
        db.commit()

        # Close the cursor and database connection
        mycursor.close()
        db.close()

    except Error:
        print(Error, "encountered during inserting data into SQL Database")     

In [4]:
# Connect to the MySQL server
db = mysql.connector.connect(
    host = "127.0.0.1",
    user = "root",
    password = "root"
)
mycursor = db.cursor()

# Create the profiles database if it doesn't exist
mycursor.execute("CREATE DATABASE IF NOT EXISTS profiles")

In [5]:
# Connect to the profiles database
db = mysql.connector.connect(
    host = "127.0.0.1",
    user = "root",
    password = "root",
    database = "profiles"
)
mycursor = db.cursor()

In [6]:
# Create the table to store the Twitter profiles data
query = "CREATE TABLE twitter_profiles (username VARCHAR(50), bio TEXT, following_count VARCHAR(10), followers_count VARCHAR(10), location VARCHAR(50), website VARCHAR(100))"
mycursor.execute(query)

In [7]:
# Reading the Twitter URLs from a CSV file:
df = pd.read_csv('twitter_links.csv', header=None)
twitter_urls = df[0]

# Scraping the profiles by calling scrape_profile function
for url in twitter_urls:
    scrape_profile(url)

C:\Users\akash\AppData\Local\Temp\ipykernel_29752\3570449315.py:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # Run the Browser in headless mode


GTN UK
@GTNUK1
WhatsApp
@WhatsApp
A & A Customs Broker
@aacb_CBPTrade
A&A Customs Brokers
@aacbdotcom
A&A Window Products
@AAWindowPRODUCT
A & B Kia
@AandB_Kia
A&B Home
@ABHomeInc
A & B Reprographics
@Abrepro
N/A
A & C CHRISTOFI LTD
@ACChristofiLtd
A&E Clothing
@aeclothing1
N/A
A & E Technologies
@AETechnologies1
Wix
@Wix
A & G Insurance LLC
@AGInsuranceLLC
